In [1]:
import os,sys, json,re, pickle
import magic, hashlib,  traceback ,ntpath, collections ,lief
from capstone import *
from capstone.x86 import *
import torch.nn as nn
import lief
from elftools.elf.elffile import ELFFile
from transformers import AdamW,AutoTokenizer
from tqdm import tqdm  # for our progress bar
from sklearn.metrics import precision_recall_fscore_support , accuracy_score,f1_score, confusion_matrix,mean_squared_error, mean_absolute_error, r2_score
from numpy import *
from num2words import num2words
import pandas as pd
from collections import defaultdict

In [2]:
MAX_ITERATIONS = 10000  # Prevent infinite looping

In [3]:
# BIN_FILE_TYPE = 'PE' #or ELF
# bin_path = '/home/raisul/DATA/temp/x86_pe_msvc_O2_static/'
# bin_files = [os.path.join(bin_path, f) for f in os.listdir(bin_path) if f.endswith(".exe")]
# ground_truth_path ='/home/raisul/DATA/temp/ghidra_x86_pe_msvc_O2_debug/'  


BIN_FILE_TYPE = 'ELF'
bin_path = '/home/raisul/DATA/x86_O2_d4/' #/home/raisul/DATA/temp/x86_pe_msvc_O2_static/'
bin_files = [f for f in os.listdir(bin_path) ]
ground_truth_path ='/home/raisul/ANALYSED_DATA/ghidra_x86_O2_d4/'  

In [4]:

def get_ground_truth_ghidra(exe_path, text_section_offset , text_section_len):

    text_sextion_end = text_section_offset + text_section_len
    
    elf_file_name = os.path.basename(exe_path)
    ghidra_file_path = os.path.join(ground_truth_path, elf_file_name.split('.')[0]) + '.json'
    
    with open(ghidra_file_path, "r") as file:
        ghidra_data = json.load(file)

    ground_truth_offsets = list(ghidra_data.keys())

    ground_truth_offsets = [int(i) for i in ground_truth_offsets]
    ground_truth_offsets = [x for x in ground_truth_offsets if text_section_offset <= x <= text_sextion_end]
    ground_truth_offsets.sort()
    return ground_truth_offsets



def find_data_in_textsection(ground_truth_offsets , text_section_offset , text_section_len, offset_inst_dict):
    data_offsets = []
    for i in range(1, len(ground_truth_offsets)-1):
        distance = ground_truth_offsets[i+1] - ground_truth_offsets[i]

        inst_len = offset_inst_dict[ground_truth_offsets[i]].size 
        
        if distance!=inst_len:
            # print('offset_ranges[i]: ',ground_truth_offsets[i] , 'offset_ranges[i-1]: ',ground_truth_offsets[i-1], ' inst_len: ',inst_len  )
            # print(ground_truth_offsets[i],' ' ,hex(ground_truth_offsets[i]) , offset_inst_dict[ground_truth_offsets[i]], ' len',offset_inst_dict[ground_truth_offsets[i]].size )
            # print("\nByte GAP ###### ",distance ,' Missing bytes: ', distance - inst_len)
            
            for j in range( ground_truth_offsets[i] +inst_len , ground_truth_offsets[i+1]  ):
                data_offsets.append(j)
                # if offset_inst_dict[j]:
                #     print("# ",j, offset_inst_dict[j].mnemonic, offset_inst_dict[j].op_str , 'inst len:',offset_inst_dict[j].size )
                # else:
                #     print("# ",j, " invalid ")
            # print('\n')
        else:
            # print(ground_truth_offsets[i],' ', hex(ground_truth_offsets[i]) , offset_inst_dict[ground_truth_offsets[i]].mnemonic,offset_inst_dict[ground_truth_offsets[i]].op_str ,' len',offset_inst_dict[ground_truth_offsets[i]].size)
            pass
    return data_offsets
    

def linear_sweep(offset_inst , target_offset):
    inst_sequence = ''
    address_list = []
    
    current_offset = target_offset
    for q in range(MAX_SEQUENCE_LENGTH):

        if current_offset in offset_inst: #if end of text section
            current_instruction = offset_inst[current_offset]
            if current_instruction is None:
                return  None
                
            current_offset = current_offset + current_instruction.size
            inst_sequence+= str( hex(current_instruction.address)) +" "+ current_instruction.mnemonic +' '+ current_instruction.op_str+ ' ; ' 
            address_list.append(current_instruction.address)
            
            if current_instruction.mnemonic in ["ret", "jmp"]: #break linear sweep
                break
                

    return inst_sequence, address_list
    

In [5]:


for bin_file_path in bin_files[1:2]:#[1:1000]:

    if BIN_FILE_TYPE == "ELF":
        bin_file_path = os.path.join(bin_path , bin_file_path)
    
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    md.detail = True
    offset_inst = {}

    
    with open(bin_file_path, 'rb') as f:

        
        try:
            if BIN_FILE_TYPE == "ELF":
                elffile = ELFFile(f)
                textSection = elffile.get_section_by_name('.text').data()
                text_section_offset = elffile.get_section_by_name('.text')['sh_offset']
              
            elif BIN_FILE_TYPE == "PE":

                        
                pe_file = lief.parse(bin_file_path)
                text_section = pe_file.get_section(".text")
                text_section_offset = text_section.pointerto_raw_data
                textSection = bytes(text_section.content)
                
            ground_truth_offsets = get_ground_truth_ghidra(bin_file_path, text_section_offset , len(textSection))
            # if len(ground_truth_offsets)<5000:
            #     print(len(ground_truth_offsets))
            #     continue
            # else:
            #     print('found')
        except Exception as e:
            print("An error occurred:", e ,bin_file_path)
            continue

    inst_sizes = {}
    for byte_index in range(len(textSection)):
        try:    

            instruction = next(md.disasm(textSection[byte_index: byte_index+15 ], text_section_offset + byte_index ), None)
            offset_inst[text_section_offset+byte_index] = instruction
            inst_sizes [text_section_offset+byte_index] = instruction.size if instruction else None
            
            # if instruction:
            #     print("%d:\t%s\t%s _\t%x" %(int(instruction.address), instruction.mnemonic, instruction.op_str, instruction.size))
                
                
            # else:
            #     print("%d:\t%s " % (text_section_offset + byte_index  , 'invalid instruction') )



                    

        except Exception as e:
            print(traceback.print_exc() )
            print(e)

    
    
    offset_inst_dict = collections.OrderedDict(sorted(offset_inst.items()))

    DATA_OFFSETS = find_data_in_textsection(ground_truth_offsets , text_section_offset , len(textSection) , offset_inst)


    code_boundary = text_section_offset+len(textSection)
    break

disasm  = offset_inst_dict

In [6]:
min_offset, max_offset = list(disasm.keys())[0] , list(disasm.keys())[-1] 
print(min_offset, max_offset )

4256 5291


In [7]:
# CONTROL_GROUPS = {
#     CS_GRP_JUMP,
#     CS_GRP_CALL,
#     CS_GRP_RET,
#     CS_GRP_IRET,
# }

# for key,val in disasm.items():
#     # print(val.groups)
#     if val == None:
#         continue
#     for group in val.groups:
#         if group in CONTROL_GROUPS:
#             print(f"0x{val.address:x}: {val.mnemonic} {val.op_str}")

In [8]:
# CONTROL_GROUPS ={"CALL", "COND_BR", "UNCOND_BR", "RET"}
CONTROL_GROUPS = {
    CS_GRP_JUMP,
    CS_GRP_CALL,
    CS_GRP_RET,
    CS_GRP_IRET,
}
BRANCH_GROUPS = {
    CS_GRP_CALL,       # Function call instruction
    CS_GRP_JUMP       # Conditional and unconditional branches
}

def _compute_destinations(disasm):
    """ Compute successor addresses (CFG) and ensure function epilogues are correctly identified. """
    dests, preds = {}, defaultdict(list)
    last_offset = list(disasm.keys())[-1]
    first_offset = list(disasm.keys())[0]

    for offset, details in disasm.items():
        if details==None:
            continue
        inst_str = details.mnemonic +' ' + details.op_str
        next_offset = offset + details.size



        if not set(details.groups) & CONTROL_GROUPS:
            # Default fallthrough for non-control flow instructions
            if next_offset <= last_offset:
                dests[offset] = [next_offset]
                # preds[next_offset].append(offset)
            else:
                dests[offset] = []
        else: #control instruction
            #unconditional jump
            if details.id == X86_INS_JMP and details.operands and details.operands[0].type == CS_OP_IMM:
                 # Unconditional jump
                op_value = details.operands[0].imm
                if op_value>=first_offset and op_value<=last_offset:
                    dests[offset] = [op_value]
                    # preds[op_value].append(offset)
            
            # elif "COND_BR" in details.groups or "CALL" in details.groups:
            elif (CS_GRP_JUMP in details.groups or CS_GRP_CALL in details.groups) :
                if details.operands and details.operands[0].type == CS_OP_IMM:
                    jump_target = details.operands[0].imm
    
                    if next_offset<=last_offset:
                        dests[offset] = [next_offset]
                        # preds[next_offset].append(offset)
                                     
                    if jump_target>=first_offset and jump_target<=last_offset and jump_target!=next_offset:
                        if offset in dests:
                            dests[offset].append(jump_target)
                        else:
                            dests[offset] = [jump_target]
        
            else:
                # print('>>>>>  ',offset, ' : ' ,inst_str)
                dests[offset] = None

        if offset in dests:
            if dests[offset] is not None:
                for target in dests[offset]:
                    preds[target].append(offset)

    return dests, preds

cfg, preds = _compute_destinations(disasm)
cfg = dict(sorted(cfg.items()))
preds = dict(sorted(preds.items()))

# def has_duplicates(lst):
#     return len(lst) != len(set(lst))

# for offset ,inst in disasm.items():
#     if inst:
#         print(offset," : ", hex(offset), ' ' ,inst.mnemonic +' ' + inst.op_str , '   ',inst.size)
#         if offset in cfg:
#             print('cfg ',cfg[offset])
#         if offset in preds:
#             print('pred ',preds[offset])

                



In [9]:


def _compute_occlusion(disasm):
    """ Identify overlapping instructions and remove """
    occlusion = defaultdict(list)
    valid_instructions = set()

    for offset, details in disasm.items():
        if details!= None:
            for i in range(offset + 1, offset + details.size):
                occlusion[i].append(offset)

    # fix nahid
    covered = set()
    for offset in sorted(disasm.keys()):
        if disasm[offset] is None:
            continue
        if offset in covered:
            # print(f"Skipping {offset} due to occlusion")
            continue  # Skip if another instruction already claimed this byte

        valid_instructions.add(offset)
        for i in range(offset, offset + disasm[offset].size):
            covered.add(i)  # Mark all bytes of this instruction as covered

    # print(f"Final valid instructions after occlusion: {sorted(valid_instructions)}")
    return occlusion, valid_instructions
occlusion_space, valid_instructions =_compute_occlusion(disasm)


In [10]:
# valid_instructions

In [11]:

# ALL_RD_CFG = {}


                
# def get_recursive_descent_cfg(disasm):
#     debug = False
#     global ALL_RD_CFG

#     for offset in disasm:
#         if debug:
#             print('\n-------------------------------------------------------\n')
#         RD_CFG = []
#         current = offset
#         while True:

#             print('current: ',current)

#             #prevent cycle
#             if current in RD_CFG:
#                 break
#             #out of bound
#             if current not in disasm:
#                 break
#             #current invalid
#             if disasm[current] is None:
#                 break
#             #return
#             if disasm[current].mnemonic in ["ret"]:
#                 break
            

                
#             #non control instruction
#             if len(set(disasm[current].groups) & CONTROL_GROUPS) ==0: 


#                 next =  current + disasm[current].size 


#             #control instruction
#             else:
#                 #jumps of calls with target
#                 if len(set(disasm[current].groups) & BRANCH_GROUPS): 
#                     if current in cfg:
#                         next = cfg[current][0]
#                     #tidi find better soltn
#                     else:
#                         next =  current + disasm[current].size 
#                 #indirect control flow
#                 else:
#                     next =  current + disasm[current].size 

#             if next not in disasm:
#                 break
#             RD_CFG.append(next)
#             current = next 
            
#         #save         
#         ALL_RD_CFG [offset] = RD_CFG





# get_recursive_descent_cfg(disasm)





In [12]:

ALL_RD_CFG = {}


                
def get_recursive_descent_cfg(disasm):
    debug = False
    global ALL_RD_CFG

    for offset in disasm:
        if debug:
            print('\n-------------------------------------------------------\n')
        RD_CFG = []
        current = offset
        while True:

            print('current: ',current)

            #prevent cycle
            if current in RD_CFG:
                break
            #out of bound
            if current not in disasm:
                break
            #current invalid
            if disasm[current] is None:
                break
            #return
            # if disasm[current].mnemonic in ["ret"]:
            #     break
            

            next =  current + disasm[current].size 


            if next not in disasm:
                break
            RD_CFG.append(next)
            current = next 
            
        #save         
        ALL_RD_CFG [offset] = RD_CFG





get_recursive_descent_cfg(disasm)





current:  4256
current:  4260
current:  4257
current:  4258
current:  4259
current:  4260
current:  4260
current:  4261
current:  4261
current:  4266
current:  4262
current:  4264
current:  4263
current:  4265
current:  4264
current:  4266
current:  4265
current:  4268
current:  4266
current:  4273
current:  4267
current:  4273
current:  4268
current:  4269
current:  4269
current:  4271
current:  4270
current:  4272
current:  4271
current:  4273
current:  4272
current:  4275
current:  4273
current:  4281
current:  4274
current:  4281
current:  4275
current:  4276
current:  4276
current:  4281
current:  4277
current:  4279
current:  4278
current:  4281
current:  4279
current:  4281
current:  4280
current:  4284
current:  4281
current:  4290
current:  4282
current:  4290
current:  4283
current:  4290
current:  4284
current:  4286
current:  4285
current:  4290
current:  4286
current:  4288
current:  4287
current:  4289
current:  4288
current:  4290
current:  4289
current:  4292
current:  

In [13]:


ALL_RD_PRED ={}
def get_recursive_preds(disasm):
    global ALL_RD_PRED

    #todo fix nahid
    for offset in disasm:
        ALL_RD_PRED[offset] = []

    for offset in disasm:
        for target in ALL_RD_CFG[offset] :
            if target in disasm: #last byte
                if target not in ALL_RD_PRED[target] :
                    ALL_RD_PRED[target].append(offset)

get_recursive_preds(disasm)

In [14]:

H_list = {}

res, data_prob,  = {}, {}
cfg, preds

min_offset, max_offset
BOTTOM = None

for offset in range(min_offset, max_offset + 1):

        if disasm[offset] is  None:
            data_prob[offset] =  1.0
        # elif offset in valid_instructions:
        #     data_prob[offset] =  0.9
        else:
            data_prob[offset] = BOTTOM

        H_list[offset] = []



In [15]:

NEAR_JUMP = 0.00001525902  # (2^32 - 1)^-1
REL_JUMP = 0.00392156862  # (2^16 - 1)^-1
DEF_USE = 1/16
def _hint_one(offset, disasm):

    """ Implements Control Flow Convergence hint. """
    debug = False

    if offset not in preds:
        return

    branches = [prev for prev in preds[offset] if set(disasm[prev].groups) & BRANCH_GROUPS]

    if disasm[offset]:
        if debug:
            print( '#  ' if offset in ground_truth_offsets else '   ', \
                  ' $ ' if len(set(disasm[offset].groups) & BRANCH_GROUPS) else '   ', \
                  ' ',offset , ' : ', hex(offset),' ',disasm[offset].mnemonic +' ' + disasm[offset].op_str  , ' ' , disasm[offset].size)

    if len(branches)<2:
        return

    if debug:
        print(branches)
    for branch in branches:
        # H[branch].add(("1rel" if disasm[branch].size == 2 else "1near", offset))

        jump_len = disasm[branch].operands[0].imm - (disasm[branch].address + disasm[branch].size)
        
        if -128 <= jump_len <= 127:
            H_list[branch].append( REL_JUMP )
            H_list[offset].append( REL_JUMP) 
        else:
            H_list[branch].append( NEAR_JUMP)
            H_list[offset].append( NEAR_JUMP )
        if debug:
            print('$$$ len', jump_len)
        


def _hint_two(offset, disasm):

    """ Implements Control Flow Crossing hint. """
    debug = False
    
    if debug:
        # print( '#' if offset in ground_truth_offsets else ' ',' ',offset , ' : ', hex(offset),' ',disasm[offset].mnemonic +' ' + disasm[offset].op_str  , ' ' , disasm[offset].size)
        print( '#  ' if offset in ground_truth_offsets else '   ', \
              ' $ ' if len(set(disasm[offset].groups) & BRANCH_GROUPS) else '   ', \
              ' ',offset , ' : ', hex(offset),' ',disasm[offset].mnemonic +' ' + disasm[offset].op_str  , ' ' , disasm[offset].size)
    inst2_offset = offset
    if not CS_GRP_JUMP in disasm[inst2_offset].groups: #inst2 have to be a control one
        return
    
    
    inst2_size   = disasm[inst2_offset].size
    inst3_offset =  inst2_offset + inst2_size

    if inst3_offset not in preds: #inst 3 has to be a target of inst1
        return

    inst3_preds_list = preds[inst3_offset]

    if inst2_offset in inst3_preds_list:
        inst3_preds_list.remove(inst2_offset)

    for inst1_offset in inst3_preds_list:
        if CS_GRP_JUMP in disasm[inst1_offset].groups: #ins1 has to be a control flow instruction

            inst1_jump_len = disasm[inst1_offset].operands[0].imm - (disasm[inst1_offset].address + disasm[inst1_offset].size)
            inst2_jump_len = disasm[inst2_offset].operands[0].imm - (disasm[inst2_offset].address + disasm[inst2_offset].size)

            

            if -128 <= inst1_jump_len <= 127:  
                hint_type_inst1 = NEAR_JUMP
            else:
                hint_type_inst1 = REL_JUMP

            if -128 <= inst2_jump_len <= 127:  
                hint_type_inst2 = NEAR_JUMP
            else:
                hint_type_inst2 = REL_JUMP

            H_list[inst1_offset].append( hint_type_inst1 )
            H_list[inst3_offset].append( hint_type_inst1 )
            H_list[inst2_offset].append( hint_type_inst2 )


            if debug:
                print('inst1: ', inst1_offset , 'inst2: ',inst2_offset, 'inst3: ',inst3_offset)
                for o in [inst1_offset , inst2_offset, inst3_offset]:
                    if o not in ground_truth_offsets:
                            print('# # # '*10 ,o)
        
        




def _hint_three(offset, disasm):

    """ Implements Register Define-Use Relation hint. """

    debug = True
    # if offset not in ground_truth_offsets:
    #     return

    
    if debug:
        print('\n\n')
        print( '#' if offset in ground_truth_offsets else ' ',' ',offset , ' : ', hex(offset),' ',disasm[offset].mnemonic +' ' + disasm[offset].op_str  , ' ' , disasm[offset].size)

        regs_read, regs_write = disasm[offset].regs_access()
        current_reads = set(disasm[offset].reg_name(r) for r in regs_read)
        current_writes = set(disasm[offset].reg_name(r) for r in regs_write)
        
        print('read'    , current_reads   )
        print('writes'  , current_writes  )
        
        # print('           current read: ',disasm[offset].regs_read , \
        #       [disasm[offset].reg_name(r) for r in disasm[offset].regs_read ] ,  \
        #       [md.reg_name(r) for r in disasm[offset].regs_read ]) #if md.reg_name(r) not in ('eflags', 'rflags')]
        # print('           current WRITE: ',  [disasm[offset].reg_name(r) for r in disasm[offset].regs_write] ) #if md.reg_name(r) not in ('eflags', 'rflags')]

    
    
    
    if offset not in preds:
        return
    
    for prev in preds[offset]:
        if debug:
            print('           prev: write: ',prev, [disasm[offset].reg_name(r) for r in disasm[prev].regs_write]) # if disasm[offset].reg_name(r) not in ('eflags', 'rflags')])

        prev_reg_write = set([ r for r in disasm[prev].regs_write  if disasm[offset].reg_name(r) not in ('eflags', 'rflags')] )
        
        curr_reg_read = set( [ r for r in disasm[offset].regs_read if disasm[offset].reg_name(r) not in ('eflags', 'rflags')])
        
        if prev_reg_write & curr_reg_read:
            H_list[prev].append(DEF_USE)
            H_list[offset].append(DEF_USE)
            
            if debug:
                print("           ----> ", prev)





for offset in disasm:
    # if offset>1200:
    #     break
    if disasm[offset] is None:
        continue

    _hint_one(offset, disasm)
    _hint_two(offset, disasm)
    _hint_three(offset, disasm)
    




#   4256  :  0x10a0   endbr64    4
read set()
writes set()



    4259  :  0x10a3   cli    1
read set()
writes set()



#   4260  :  0x10a4   push rbx   1
read {'rbx', 'rsp'}
writes {'rsp'}
           prev: write:  4256 []
           prev: write:  4259 []



#   4261  :  0x10a5   mov esi, 0x10   5
read set()
writes {'esi'}
           prev: write:  4260 ['rsp']



    4262  :  0x10a6   adc byte ptr [rax], al   2
read {'al', 'rflags', 'rax'}
writes {'rflags'}



    4263  :  0x10a7   add byte ptr [rax], al   2
read {'al', 'rax'}
writes {'rflags'}



    4264  :  0x10a8   add byte ptr [rax], al   2
read {'al', 'rax'}
writes {'rflags'}
           prev: write:  4262 ['rflags']



    4265  :  0x10a9   add byte ptr [rax - 0x7f], cl   3
read {'cl', 'rax'}
writes {'rflags'}
           prev: write:  4263 ['rflags']



#   4266  :  0x10aa   sub rsp, 0xe0   7
read {'rsp'}
writes {'rflags', 'rsp'}
           prev: write:  4261 []
           prev: write:  4264 ['rflags']



    4267  :  0x10ab  

In [16]:

H={}
def update_H():
    """ MATH determined from https://www.cs.purdue.edu/homes/zhan3299/res/ICSE19.pdf
    """
    global H
    for offset in H_list:
        
        prod = 1.0
        if len(H_list[offset])>0:
            for hint in H_list[offset]:
                prod = prod * hint
            H[offset] = prod
        else:
            H[offset] = BOTTOM
update_H()

In [17]:
RH = {}

for offset, inst in disasm.items():
    RH[offset] = set()

import math
def safe_product(numbers):
    res = None
    if len(numbers)==1:
        res =  numbers[0]
    else:
        log_sum = sum(math.log(x) for x in numbers)
        res = math.exp(log_sum)
    if res == 0:
        return 2.2250738585072014e-307
    return res
    

def calc_data_prob(offset):
    global RH
    d=1.0
    factors = []
    for rh in RH[offset]:
        factors.append(H[rh])
    if len(factors):
        d = safe_product(factors)
    return d


def _forward_propagation(disasm, data_prob, cfg):
    """ Iterative analyais to find instructions that lead to bad assembly.
        Outside of control flow, this is guarenteed to be data

        attempting to write ~line 10 algorithm 1 of https://www.cs.purdue.edu/homes/zhan3299/res/ICSE19.pdf
    """
    global RH
    debug = False
    fixed_point = True
    for offset, inst in disasm.items():
        if data_prob[offset] == 1.0:
            # Already know instruction is data
            continue
        if disasm[offset] is None:
            continue

        # line 13-15
        # update this instructions probability
        if H[offset] and offset not in RH[offset]:
            RH[offset].add(offset)
            
            # data_prob[offset] = 1.0
            data_prob[offset] = calc_data_prob(offset)
            # for h in RH[offset]:
            #     data_prob[offset] *= H[h]
                
            if 0 in data_prob.values():
                print('here1')
                return
        #line 16-20
        for n in ALL_RD_CFG[offset]:
            diff = set(RH[offset]) - set(RH[n])
            if len(diff):
                RH[n] = RH[n] | diff
                data_prob[n] = calc_data_prob(n)
                
                if n<offset:
                    fixed_point = False
    return fixed_point

# old_RH = RH.copy()
# _forward_propagation(disasm, data_prob, cfg)
# for i in RH:
#     if len(RH[i] - old_RH[i]):
#         print(RH[i] - old_RH[i])

In [18]:


import builtins
def _adjust_occlusion_probs(disasm, data_prob, occlusion_space):
    """ attempting to write ~line 22 algorithm 1 of https://www.cs.purdue.edu/homes/zhan3299/res/ICSE19.pdf

        Struggled with which probabilities should be adjusted and whether that is a global change
        or incremental


    """
    for offset, detail in disasm.items():
        if not data_prob[offset] == BOTTOM:
            # Only update if data probability is unknown
            continue

        
        # Find probability of being data for each overlapping instruction
        occluded_probs = []
        for j in occlusion_space[offset]:
            if data_prob[j] == BOTTOM : #todo nahid hack to prevent zero data prob
                continue
            occluded_probs.append(data_prob[j])

        print('here', offset)
        if len(occluded_probs)==0:
            # If not overlapping instructions, leave probability as is
            continue
        print('here2', offset , occluded_probs)
        # Step II. In lines 22-24, the algorithm traverses all the addresses
        # and performs local propagation of probabilities within occlusion space of individual instructions. Particularly, for each
        # address i, it finds its occluded peer j that has the minimal
        # probability (i.e., the most likely instruction). The likelihood
        # of i being data is hence computed as 1 − D[j] (line 24).
        new_occluded_prob = 1 - builtins.min(occluded_probs) # nahid fix*0.9
        print(data_prob[offset] , new_occluded_prob)
        if new_occluded_prob==0:
            data_prob[offset] = 0.1 #2.2250738585072014e-307
            print('occlud' ,offset)
        else:
            data_prob[offset] = new_occluded_prob 
        
        # if (data_prob[offset] ) ==0:
        #     print(1 - builtins.min(occluded_probs) , builtins.min(occluded_probs) ,occluded_probs)
        #     print('zero zero')
        # print("adjusting {} to {}".format(offset, data_prob[offset])) #todo uncomment

_adjust_occlusion_probs(disasm, data_prob, occlusion_space)

here 4256
here 4259
here 4260
here 4261
here 4262
here 4263
here 4264
here 4265
here 4266
here 4267
here 4268
here 4269
here 4270
here 4271
here 4272
here 4273
here 4274
here 4275
here 4276
here 4277
here 4278
here 4279
here 4280
here 4281
here 4282
here 4283
here 4284
here 4285
here 4286
here 4287
here 4288
here 4289
here 4290
here 4291
here 4292
here 4293
here 4294
here 4295
here 4296
here 4297
here 4298
here 4299
here 4300
here 4301
here 4302
here 4303
here 4304
here 4305
here 4306
here 4307
here 4308
here 4309
here 4310
here 4311
here 4312
here 4313
here 4314
here 4315
here 4316
here 4317
here 4318
here 4319
here 4322
here 4323
here 4324
here 4325
here 4326
here 4327
here 4328
here 4329
here 4330
here 4331
here 4332
here 4333
here 4334
here 4335
here 4336
here 4337
here 4338
here 4339
here 4340
here 4341
here 4342
here 4343
here 4344
here 4345
here 4346
here 4347
here 4348
here 4349
here 4351
here 4352
here 4353
here 4354
here 4355
here 4356
here 4357
here 4358
here 4359
here 4360


In [19]:
# def _back_propagation(disasm):
#     """ Iterative analysis to find instructions that lead to bad assembly.
#         Outside of control flow, this is guaranteed to be data.

#         Attempting to implement ~line 25 of Algorithm 1 from:
#         https://www.cs.purdue.edu/homes/zhan3299/res/ICSE19.pdf
#     """
#     fixed_point = True
#     for i in data_prob.keys():
#         if data_prob[i] == BOTTOM or i not in preds:
#             # Cannot propagate unknown probability or unknown predecessors
#             continue


#         for p in preds[i]:
#             # Updated probability propagation logic


#             if data_prob[p] == BOTTOM or data_prob[p] < data_prob[i]:
#                 data_prob[p] = data_prob[i] 
#                 fixed_point = False  # Mark that we've updated a probability

#                 if p > i:
#                     # Ensure continued processing if new updates occur
#                     fixed_point = False


#     return fixed_point



In [20]:
def _back_propagation(disasm):
    """ Iterative analysis to find instructions that lead to bad assembly.
        Outside of control flow, this is guaranteed to be data.

        Attempting to implement ~line 25 of Algorithm 1 from:
        https://www.cs.purdue.edu/homes/zhan3299/res/ICSE19.pdf
    """
    fixed_point = True
    for i in data_prob.keys():
        if data_prob[i] == BOTTOM or i not in ALL_RD_PRED:
            # Cannot propagate unknown probability or unknown predecessors
            continue


        for p in ALL_RD_PRED[i]:
            # Updated probability propagation logic


            if data_prob[p] == BOTTOM or data_prob[p] < data_prob[i]:
                data_prob[p] = data_prob[i] 
                fixed_point = False  # Mark that we've updated a probability

                if p > i:
                    # Ensure continued processing if new updates occur
                    fixed_point = False


    return fixed_point

# 

In [34]:
for _ in range(30): #MAX_ITERATIONS
    if _%100==0:
        print('oter',_)

    fixed_point = True
    fixed_point = _forward_propagation(disasm, data_prob, cfg) and fixed_point
    fixed_point = _adjust_occlusion_probs(disasm, data_prob, occlusion_space) and fixed_point
    # fixed_point = _back_propagation(disasm) and fixed_point
    
    if fixed_point is True:
        break

oter 0
here 4256
here 4259
here 4260
here 4261
here 4262
here 4263
here 4264
here 4265
here 4266
here 4267
here 4268
here 4269
here 4270
here 4271
here 4272
here 4273
here 4274
here 4275
here 4276
here 4277
here 4278
here 4279
here 4280
here 4281
here 4282
here 4283
here 4284
here 4285
here 4286
here 4287
here 4288
here 4289
here 4290
here 4291
here 4292
here 4293
here 4294
here 4295
here 4296
here 4297
here 4298
here 4299
here 4300
here 4301
here 4302
here 4303
here 4304
here 4305
here 4306
here 4307
here 4308
here 4309
here 4310
here 4311
here 4312
here 4313
here 4314
here 4315
here 4316
here 4317
here 4318
here 4319
here 4322
here 4323
here 4324
here 4325
here 4326
here 4327
here 4328
here 4329
here 4330
here 4331
here 4332
here 4333
here 4334
here 4335
here 4336
here 4337
here 4338
here 4339
here 4340
here 4341
here 4342
here 4343
here 4344
here 4345
here 4346
here 4347
here 4348
here 4349
here 4351
here 4352
here 4353
here 4354
here 4355
here 4356
here 4357
here 4358
here 4359
her

In [35]:
def is_interrupt(instr):
    return instr.mnemonic in {'int', 'int3', 'syscall', 'sysenter', 'iret', 'iretq'}
def is_nop(instr):
    return instr.mnemonic == 'nop'

In [36]:
P = {}
for offset in disasm:

    if disasm[offset] is None:
        P[offset] = 0
        continue
    if data_prob[offset] ==1.0 or data_prob[offset] is None:
        P[offset] = 0
        continue

    #padding
    if is_interrupt(disasm[offset]) or is_nop(disasm[offset]):
        P[offset] = 0
        continue

    s=1/data_prob[offset]
    
    for j in occlusion_space[offset]:
        s = s + 1/(data_prob[j] if data_prob[j] else 1)#nahid hack
    P[offset] = (1/data_prob[offset]) /s

In [37]:
predictions = []
for offset, p in P.items():

        if p >.01 :
            predictions.append(offset)
        # else:
        #     print(offset, p)
false_positive = set(predictions) -set (ground_truth_offsets)
false_negative = set (ground_truth_offsets) - set(predictions)
true_positive = set (ground_truth_offsets) & set(predictions)
print('false_positive: ',len(false_positive) , ' false_negative: ',len(false_negative) , ' true_positive: ',len(true_positive), 'total:', len(ground_truth_offsets)) 


false_positive:  0  false_negative:  39  true_positive:  206 total: 245


In [38]:
# false_positive:  0  false_negative:  70  true_positive:  175 total: 245

In [39]:
# for offset in disasm:

#     # try:
#         tok = "   "
#         if offset in false_negative:
#             tok = "N-N"
#         elif offset in false_positive:
#             tok = "x  "
#         if disasm[offset]:
#             print( str(format(P[offset],".15f" )).ljust(10),'  ',str( format(data_prob[offset],".15f" ) ).ljust(10), str(H[offset]).ljust(10) ,str(RH[offset]).ljust(20) ,tok, '#' if offset in ground_truth_offsets else ' ',' ',offset , ' : ', hex(offset),' ',disasm[offset].mnemonic +' ' + disasm[offset].op_str  , ' ' , disasm[offset].size)
#         else:
#             print( str(format(P[offset],".15f" )).ljust(10),'  ',str( format(data_prob[offset],".15f" ) ).ljust(10), str(H[offset]).ljust(10) ,str(RH[offset]).ljust(20) , tok,'   ', offset , ' : '    , 'invalid instruction') 

#     # except Exception as e:
#     #     print(traceback.print_exc() )
#     #     print(e)

In [27]:
for offset in disasm:

    # try:
        tok = "   "
        if offset in false_negative:
            tok = "N-N"
        elif offset in false_positive:
            tok = "x  "
        if disasm[offset]:
            print( str(format(P[offset],".15f" )).ljust(20),'  ',str( (data_prob[offset] ) ).ljust(20), str(H[offset]).ljust(10) ,str(RH[offset]).ljust(20) ,tok, '#' if offset in ground_truth_offsets else ' ',' ',offset , ' : ', hex(offset),' ',disasm[offset].mnemonic +' ' + disasm[offset].op_str  , ' ' , disasm[offset].size)
        else:
            print( str(format(P[offset],".15f" )).ljust(20),'  ',str( (data_prob[offset]) ).ljust(20), str(H[offset]).ljust(10) ,str(RH[offset]).ljust(20) , tok,'   ', offset , ' : '    , 'invalid instruction') 

    # except Exception as e:
    #     print(traceback.print_exc() )
    #     print(e)

0.000000000000000       None                 None       set()                N-N #   4256  :  0x10a0   endbr64    4
0.000000000000000       1.0                  None       set()                        4257  :  invalid instruction
0.000000000000000       1.0                  None       set()                        4258  :  invalid instruction
0.000000000000000       None                 None       set()                        4259  :  0x10a3   cli    1
0.000000000000000       None                 None       set()                N-N #   4260  :  0x10a4   push rbx   1
0.000000000000000       None                 None       set()                N-N #   4261  :  0x10a5   mov esi, 0x10   5
0.000000000000000       None                 None       set()                        4262  :  0x10a6   adc byte ptr [rax], al   2
0.000000000000000       None                 None       set()                        4263  :  0x10a7   add byte ptr [rax], al   2
0.000000000000000       None                 No

In [28]:
predictions = list(valid_instructions)
false_positive = set(predictions) -set (ground_truth_offsets)
false_negative = set (ground_truth_offsets) - set(predictions)
true_positive = set (ground_truth_offsets) & set(predictions)
print('false_positive: ',len(false_positive) , ' false_negative: ',len(false_negative) , ' true_positive: ',len(true_positive), 'total:', len(ground_truth_offsets)) 
print(predictions.count(0))

false_positive:  12  false_negative:  1  true_positive:  244 total: 245
0


In [29]:
# ALL_RD_CFG[4266]